# Domain adaptation

In [ ]:
from pathlib import Path
DATASET_ZIP_PATH = '/tmp/Adaptiope.zip'
DATASET_EXTRACTION_PATH = '/tmp/Adaptiope'
DATASET_PATH = Path('./data/adaptiope_small')

## Dataset extraction

In [ ]:
from os import makedirs
from os.path import join, isdir
from shutil import copytree

In [ ]:
!mkdir -p {DATASET_EXTRACTION_PATH}
!unzip -d {DATASET_EXTRACTION_PATH} {DATASET_ZIP_PATH}

In [ ]:
if not isdir(DATASET_PATH):
    classes = ["backpack", "bookcase", "car jack", "comb", "crown", "file cabinet", "flat iron", "game controller", "glasses",
               "helicopter", "ice skates", "letter tray", "monitor", "mug", "network switch", "over-ear headphones", "pen",
               "purse", "stand mixer", "stroller"]
    for d, td in zip([
        f"{DATASET_EXTRACTION_PATH}/Adaptiope/product_images",
        f"{DATASET_EXTRACTION_PATH}/Adaptiope/real_life"],[
        f"{DATASET_PATH}/product_images",
        f"{DATASET_PATH}/real_life"]):
        makedirs(td)
        for c in classes:
            c_path = join(d, c)
            c_target = join(td, c)
            copytree(c_path, c_target)

## Dataset exploration

In [ ]:
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt

In [ ]:
dataset = ImageFolder(DATASET_PATH / "product_images")
idx_to_class = {v: k for k,v in dataset.class_to_idx.items()}

In [ ]:
# imgs: List[str, int] path, class
seen_classes = set()
imgs = []
for i, (p, c) in enumerate(dataset.imgs):
    if c not in seen_classes:
        seen_classes.add(c)
        imgs.append(i)

In [ ]:
fig, axs = plt.subplots(2, 5, figsize=(10,10))
for i in range(2):
    for j in range(5):
        image, title = dataset[imgs.pop(0)]
        axs[i,j].imshow(image)
        axs[i,j].set_title(idx_to_class[title])
plt.show()        

## Utility functions

In [ ]:
import random
import numpy as np
import torch

from IPython.display import clear_output

import matplotlib.pyplot as plt
%matplotlib inline

def set_random_seed(seed=0) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def get_device() -> torch.device:
    return torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def print_epoch_chart(
    train_losses,
    val_losses,
    train_acc,
    val_acc,
    epoch,
    num_epochs
):
    clear_output(wait=True)
    
    fig, (loss_ax, acc_ax) = plt.subplots(1, 2, figsize=(15,7))
    x = np.arange(len(train_losses))
    loss_ax.plot(x, train_losses, label='train')
    loss_ax.plot(x, val_losses, label='validation')
    loss_ax.set_title("Loss")
    loss_ax.set_ylim([0, 5])
    loss_ax.legend()
    acc_ax.plot(x, train_acc, label='train')
    acc_ax.plot(x, val_acc, label='validation')
    acc_ax.set_title("Accuracy")
    acc_ax.legend()
    fig.suptitle(f"Epoch {epoch+1} of {num_epochs}")
    plt.show()

## Losses

In [ ]:
def coral(source_extracted_features, target_extracted_features, **kwargs):
    source = source_extracted_features.double()
    target = target_extracted_features.double()
    
    d = source.data.shape[1]
    ns, nt = source.data.shape[0], target.data.shape[0]
    # source covariance
    xm = torch.mean(source, 0, keepdim=True) - source
    xc = xm.t() @ xm / (ns - 1)

    # target covariance
    xmt = torch.mean(target, 0, keepdim=True) - target
    xct = xmt.t() @ xmt / (nt - 1)

    # frobenius norm between source and target
    loss = torch.mul((xc - xct), (xc - xct))
    loss = torch.sum(loss) / (4*d*d)
    return loss

## Contrastive Domain Discrepancy

$\require{color}$
*Contrative Domain Discrepancy (CDD)* is a measure used to quantify the dissimilarity between classes between source and destination target. The computation is made on features extracted from linear layers of the network after the pretrained backbone module.

TODO: WRITE EQUATION FROM PAPER

Let's suppose the two following table contains the output of the two linear layers for the source dataset with a batch size equal to 5 and, an output size equal to 5 for the first layer and to 3 for the latter.
$$
\begin{array}{|c|c|c|c|c|}
    \hline
    0.496257 & 0.768222 & 0.0884774 & 0.13203  & 0.307423 \\
    \hline
    0.634079 & 0.490093 & 0.896445  & 0.455628 & 0.632306 \\
    \hline
    0.348893 & 0.401717 & 0.0223258 & 0.168859 & 0.293888 \\
    \hline
    0.518522 & 0.697668 & 0.800011  & 0.161029 & 0.282269 \\
    \hline
    0.681609 & 0.915194 & 0.3971    & 0.874156 & 0.419408 \\
    \hline
\end{array}\quad\quad
\begin{array}{|c|c|c|c|c|}
    \hline
    0.552907 & 0.952738  & 0.0361648 \\
    \hline
    0.185231 & 0.373417  & 0.3051    \\
    \hline
    0.932    & 0.17591   & 0.269834  \\
    \hline
    0.15068  & 0.0317195 & 0.20813   \\
    \hline
    0.929799 & 0.723109  & 0.742336  \\
    \hline
\end{array}
$$

Same for the target dataset with a batch size equal to 4

$$
\begin{array}{|c|c|c|c|c|}
    \hline
    0.526296 & 0.243658 & 0.584592 & 0.0331526 & 0.138717 \\
    \hline
    0.242235 & 0.815469 & 0.793161 & 0.278252  & 0.481959 \\
    \hline
    0.81978  & 0.997067 & 0.698441 & 0.567546  & 0.835243 \\
    \hline
    0.205599 & 0.593172 & 0.112347 & 0.153457  & 0.241708 \\
    \hline
\end{array}\quad\quad
\begin{array}{|c|c|c|c|c|}
\hline
0.726237 & 0.70108  & 0.203824 \\
\hline
0.651054 & 0.774486 & 0.436891 \\
\hline
0.519091 & 0.615852 & 0.810188 \\
\hline
0.980097 & 0.114688 & 0.316765 \\
\hline
\end{array}
$$

For each dataset it is also required a list describing the distribution of samples for each class, for source dataset $[2,2,1]$, for target dataset $[1, 1, 2]$.

#### Distance computation
There are three types of distance that can be computed:

- distance source/source dataset
- distance target/target dataset
- distance source/target dataset

All three distances are computed for each output of linear layer aggregating the different number of features summing them (`compute_pairwise_distance` function).

Obtained result for distance source/target with shape equal to `[batch size source X batch size target]`:

$$
\begin{array}{cc}
Layer 1& Layer2 \\
\begin{array}{|c|c|c|c|c|}
\hline
 0.560438 & 0.615181 & 0.997362 & 0.120472  \\
 \hline
 0.591715 & 0.324145 & 0.384421 & 1.0529    \\
 \hline
 0.415092 & 0.82409  & 1.48532  & 0.0682521 \\
 \hline
 0.28955  & 0.143876 & 0.66175  & 0.583425  \\
 \hline
 1.29631  & 0.718872 & 0.383529 & 0.962352  \\
 \hline
\end{array}&
\begin{array}{|c|c|c|c|c|}
\hline
 0.121484 & 0.201988 & 0.713748 & 0.963555   \\
 \hline
 0.410307 & 0.395216 & 0.425351 & 0.698889   \\
 \hline
 0.3225   & 0.465132 & 0.656027 & 0.00826399 \\
 \hline
 0.779328 & 0.854408 & 0.839412 & 0.706618   \\
 \hline
 0.331919 & 0.173635 & 0.184789 & 0.553817   \\
\hline
\end{array}
\end{array}
$$

The output for source/source and target/target has respectively shape of `[batch size source X batch size source]` and `[batch size target X batch size target]`. Distances of the same dataset require an extra step that iterates over the diagonal extracting square submatrices with size length equal to the amount of samples for a specific class inside the batch size under consideration. Obtained result for one layer of source/source distance (given distribution $[2,2,1]$):

$$
\begin{array}{ccc}
Class1&Class2&Class3  \\
\begin{array}{|c|c|}
\hline
 0        & 0.543124 \\
 \hline
 0.543124 & 0        \\
\hline
\end{array}&
\begin{array}{|c|c|}
\hline
 0       & 0.63506 \\
 \hline
 0.63506 & 0       \\
\hline
\end{array}&
\begin{array}{|c|c|}
\hline
 0 \\
\hline
\end{array}
\end{array}
$$

The matrix of distance source/target is averaged over classes submatrices (colors are used to show class groups) creating a new maen matrix:

$$
\begin{array}{cc}
\text{Layer1 before the average}&\text{Layer1 after}\\
\begin{array}{|c|c|c|c|c|}
\hline
 \color{red}{0.560438} & \color{green}{0.615181} & \color{orange}{0.99736} & \color{orange}{0.120472}  \\
 \hline
 \color{red}{0.591715} & \color{green}{0.324145} & \color{orange}{0.384421} & \color{orange}{1.0529}    \\
 \hline
 \color{magenta}{0.415092 }& \color{brown}{0.82409  }& \color{olive}{1.48532  }& \color{olive}{0.0682521} \\
 \hline
 \color{magenta}{0.28955  }& \color{brown}{0.143876 }& \color{olive}{0.66175  }& \color{olive}{0.583425 } \\
 \hline
 \color{pink}{1.29631  }& \color{darkgrey}{0.718872 }& \color{teal}{0.383529 }& \color{teal}{0.962352 } \\
 \hline
\end{array}&
\begin{array}{|c|c|c|c|c|}
\hline
 \color{red}{0.511834} & \color{green}{0.476363} & \color{orange}{0.607436} & \color{orange}{0.607436} \\
 \hline
 \color{red}{0.511834} & \color{green}{0.476363} & \color{orange}{0.607436} & \color{orange}{0.607436} \\
 \hline
 \color{magenta}{0.432476 }& \color{brown}{0.397005 }& \color{olive}{0.567757 }& \color{olive}{0.567757 }\\
 \hline
 \color{magenta}{0.432476 }& \color{brown}{0.397005 }& \color{olive}{0.567757 }& \color{olive}{0.567757 }\\
 \hline
 \color{pink}{0.280219 }& \color{darkgrey}{0.307591 }& \color{teal}{0.655509 }& \color{teal}{0.655509 }\\
 \hline
\end{array}
\end{array}
$$

TODO: HIGHLIGHT THE PART OF THE EQUATION THAT IS DONE WITH THIS PROCEDURE

Same mean procedure is applied to source/source and target/target data.

#### Kernel computation
Distances and mean matrices are then used to compute a kernel for each layer, the list of results are summed up. The final matrix has a shape of `[number of samples X number of samples]` and expresses the correlation between samles belonging batch sizes (in the following table source/target table is presented):

$$
\begin{array}{|c|c|c|}
\hline
 3.92554 & 3.90333 & 3.65386 \\
 \hline
 3.52906 & 3.25892 & 4.40912 \\
 \hline
 0.74786 & 2.26064 & 4.51405 \\
 \hline
\end{array}
$$

#### Maximum Mean Discrepancy (MMD) computation
Three kernels extracted from source/source, targe/target and source/target are composed together to obtaind the MMD):

TODO: WRITE EQUATION

The result is:

$$
\begin{array}{|c|c|c|}
\hline
  8.30697 &  8.33474 & 4.42708 \\
  \hline
  9.13727 &  9.64986 & 3.07151 \\
  \hline
 18.5041  & 15.4785  & 7.26769 \\
 \hline
\end{array}
$$

where the diagonal represents the intra class discrepancy and the rest of the matrix the infra class discrepancy.

$$
\begin{array}{cc}
\text{Intra class elements}&\text{Infra class elements}\\
\begin{array}{|c|c|c|}
\hline
  8.30697 &  \color{lightgrey}{8.33474} & \color{lightgrey}{4.42708} \\
  \hline
  \color{lightgrey}{9.13727} &  9.64986 & \color{lightgrey}{3.07151} \\
  \hline
  \color{lightgrey}{18.5041}  & \color{lightgrey}{15.4785}  & 7.26769 \\
 \hline
\end{array}&
\begin{array}{|c|c|c|}
\hline
  \color{lightgrey}{8.30697} &  8.33474 & 4.42708 \\
  \hline
  9.13727 &  \color{lightgrey}{9.64986} & 3.07151 \\
  \hline
 18.5041  & 15.4785  & \color{lightgrey}{7.26769} \\
 \hline
\end{array}
\end{array}
$$

#### CDD, intra class, infra class computation
The intra and infra class measures can be computed averaging over vectors of the previous step. The CDD instead can be obtained applying the following formula:

TODO: WRITE EQUATION

In [ ]:
from typing import Tuple, List, Dict, Optional

class ConstrativeDomainDiscrepancy():
    def __init__(
        self,
        num_layers: int,
        kernel_orders: Tuple[int],
        kernel_multipliers: Tuple[int],
        num_classes: int,
        device: torch.device
    ):
        self.num_layers = num_layers
        self.kernel_orders = kernel_orders
        self.kernel_multipliers = kernel_multipliers
        self.num_classes = num_classes
        self.device = device
        
        assert num_layers == len(kernel_orders) == len(kernel_multipliers), "There must be one kernel order and multiplier for each layer"
    
    def activate(
        self,
        source_layers_out: List[torch.Tensor], # shape: [num_layers X batch_size_source X output_of_layer(i)]
        target_layers_out: List[torch.Tensor], # shape: [num_layers X batch_size_target X output_of_layer(i)]
        samples_for_class_source: List[int], # shape: [num_classes]
        samples_for_class_target: List[int], # shape: [num_classes] 
    ) -> Dict[str, float]:
        assert len(source_layers_out) == len(target_layers_out) == self.num_layers, "number of layers must be the same"

        layers_dist = []
        layers_mean = []
        for src, trg in zip(source_layers_out, target_layers_out):
            layer_distances, layer_mean_distances = self.activate_layer(
                src,
                trg,
                samples_for_class_source,
                samples_for_class_target
            )
            layers_dist.append(layer_distances)
            layers_mean.append(layer_mean_distances)
            
        for i in range(self.num_layers):
            for c in range(len(samples_for_class_source)):
                # shape goes from [num_classes] to [num_classes X 1 X 1]
                layers_mean[i]['ss'][c] = layers_mean[i]['ss'][c].view(
                    len(samples_for_class_source), 1, 1
                )
                layers_mean[i]['tt'][c] = layers_mean[i]['tt'][c].view(
                    len(samples_for_class_source), 1, 1
                )
        
        class_kernel_dist = self.classes_mean_kernel(
            samples_for_class_source,
            samples_for_class_target,
            self.compute_cum_kernel_dist(
                [i['st'] for i in layers_dist],
                [i['st'] for i in layers_mean]
            )
        )
        
        cum_kernel_dist_ss = []
        cum_kernel_dist_tt = []
        for class_index in range(len(samples_for_class_source)):
            cum_kernel_dist_ss.append(
                torch.mean(
                    self.compute_cum_kernel_dist(
                        [i['ss'] for i in layers_dist],
                        [i['ss'] for i in layers_mean],
                        class_index
                    ).view(len(samples_for_class_source), -1), dim=1
                )
            )
            cum_kernel_dist_tt.append(
                torch.mean(
                    self.compute_cum_kernel_dist(
                        [i['tt'] for i in layers_dist],
                        [i['tt'] for i in layers_mean],
                        class_index
                    ).view(len(samples_for_class_source), -1), dim=1
                )
            )
        kernel_dist_ss = torch.stack(cum_kernel_dist_ss, dim=0)
        # understand why transpose here
        kernel_dist_tt = torch.stack(cum_kernel_dist_tt, dim=0).transpose(1, 0)
        
        mmds = kernel_dist_ss + kernel_dist_tt - 2 * class_kernel_dist
        
        intra_mmds = torch.diag(mmds, 0)
        intra = torch.sum(intra_mmds) / self.num_classes
        
        inter = None
        # trick to create a mask to exclude diagonal like
        # False True True
        # True False True
        # True True False
        inter_mask = (torch.ones([len(samples_for_class_source), len(samples_for_class_source)]).to(self.device) - \
            torch.eye(len(samples_for_class_source))).type(torch.bool)
        inter_mmds = torch.masked_select(mmds, inter_mask)
        # inter is a flatten of all cells except the diagonal
        inter = torch.sum(inter_mmds) / (self.num_classes * (self.num_classes - 1))
        
        cdd = intra - inter
        return {
            'ccd': cdd,
            'intra': intra,
            'inter': inter
        }

        
    def activate_layer(
        self,
        source_features: torch.Tensor, # shape: [batch_size_source X output_of_layer(i)]
        target_features: torch.Tensor, # shape: [batch_size_target X output_of_layer(i)]
        samples_for_class_source: List[int], # shape: [num_classes]
        samples_for_class_target: List[int], # shape: [num_classes] 
    ):
        assert len(samples_for_class_source) == len(samples_for_class_target), "number of classes must be the same"
        
        num_classes = len(samples_for_class_source)
        
        # ss means source, source
        # tt means target, target
        # st means source, target
        distances = {}
        
        distances['ss'] = self.compute_pairwise_distance(source_features, source_features)
        distances['tt'] = self.compute_pairwise_distance(target_features, target_features)
        distances['st'] = self.compute_pairwise_distance(source_features, target_features)
        distances['ss'] = self.extract_classwise_information(
            distances['ss'],
            samples_for_class_source
        )
        distances['tt'] = self.extract_classwise_information(
            distances['tt'],
            samples_for_class_target
        )
        
        mean_distances = self.pairwise_class_mean_distance(
            samples_for_class_source,
            samples_for_class_target,
            distances
        )
        
        return distances, mean_distances
        
    def compute_pairwise_distance(
        self,
        ten1: torch.Tensor, # shape [batch_size_1 x features_length]
        ten2: torch.Tensor # shape [batch_size_2 x features_length]
    ) -> torch.Tensor: # shape [batch_size_1 x batch_size_2 x features_length]
        # batch_size 1 and 2 could be different
        assert ten1.shape[1] == ten2.shape[1], "Features length must be the same"
        batch_size_1 = ten1.shape[0]
        batch_size_2 = ten2.shape[0]
        
        features_length = ten1.shape[1]
        
        # goes from [batch_size_1 x features_length] to [batch_size_1 x batch_size_2 x features_length]
        expanded_1 = ten1.unsqueeze(1).expand(batch_size_1, batch_size_2, features_length)
        expanded_2 = ten2.unsqueeze(0).expand(batch_size_1, batch_size_2, features_length)
        
        # sum over the dimension 2 collases features
        # goes from [batch_size_1 x batch_size_2 x features_length] to [batch_size_1 x batch_size_2]
        squared_distance = ((expanded_1 - expanded_2)**2).sum(2)
        return squared_distance
    
    def extract_classwise_information(
        self,
        distance: torch.Tensor, # shape: [batch_size x batch_size]
        samples_for_class: List[int], # shape: [num_classes]
    ) -> List[torch.Tensor]:
        # the idea is to create one square submatrix for each class with side
        # equal to the amount of samples for that class inside the batch. In this way the
        # size of matrices will be proportional to the impact of it inside the batch_size
        square_submatrices_length = [length for length in samples_for_class]
        
        classes_distance = []
        x_offset = 0
        y_offset = 0
        for length in square_submatrices_length:
            class_distance = distance[x_offset:x_offset+length, y_offset:y_offset+length]
            classes_distance.append(class_distance)
            # translation over the diagonal of the matrix
            x_offset += length
            y_offset += length
        return classes_distance
    
    def pairwise_class_mean_distance(
        self,
        samples_for_class_source: List[int], # shape: [num_classes] 
        samples_for_class_target: List[int], # shape: [num_classes] 
        distances: Dict[str, torch.Tensor] # shape [batch_size_1 x batch_size_2 x features_length]
    ) -> Dict[str, torch.Tensor]:
        # patches will contain the square submatrices computed in extract_classwise_information
        patches = {}
        mean_distances = {}
        mean_distances['st'] = torch.zeros_like(distances['st'], requires_grad=False).to(self.device)
        mean_distances['ss'] = []
        mean_distances['tt'] = []
        for c in range(len(samples_for_class_source)):
            mean_distances['ss'] += [
                torch.zeros(
                    len(samples_for_class_source), requires_grad=False
                ).to(self.device)
            ]
            mean_distances['tt'] += [
                torch.zeros(
                    len(samples_for_class_source), requires_grad=False
                ).to(self.device)
            ]
        
        classes_distance = []
        source_start = 0
        source_end = 0
        for source_class_index in range(len(samples_for_class_source)):
            source_stard = source_end
            source_end = source_stard + samples_for_class_source[source_class_index]
            patches['ss'] = distances['ss'][source_class_index]
            
            target_start = 0
            target_end = 0
            for target_class_index in range(len(samples_for_class_target)):
                target_start = target_end
                target_end = target_start + samples_for_class_target[target_class_index]
                patches['tt'] = distances['tt'][target_class_index]
                
                patches['st'] = distances['st'][
                    source_start:source_start + samples_for_class_source[source_class_index],
                    target_start:target_start + samples_for_class_target[target_class_index]
                ]
                
                mean = self.mean_estimation(patches)
                
                mean_distances['ss'][source_class_index][target_class_index] = mean
                mean_distances['tt'][source_class_index][target_class_index] = mean
                mean_distances['st'][source_stard:source_end, target_start:target_end] = mean

        return mean_distances
    
    def mean_estimation(
        self,
        distances_patches: Dict[str, torch.Tensor]
    ) -> torch.Tensor:
        patches_sum = torch.sum(distances_patches['ss']) + \
            torch.sum(distances_patches['tt']) + \
            torch.sum(distances_patches['st'])
        
        batch_size_source = distances_patches['ss'].shape[0]
        batch_size_target = distances_patches['tt'].shape[0]
        
        # batch_size_source and batch_size_taret are removed in order
        # to obtain an unbiased estimator since total_amount_values
        # will be used as denominator for the mean computation
        total_amount_values = (batch_size_source * batch_size_source) + \
            (batch_size_target * batch_size_target) + \
            2 * (batch_size_source * batch_size_target) - \
            batch_size_source - batch_size_target
        
        mean = patches_sum.item() / total_amount_values
        return mean
    
    def compute_cum_kernel_dist(
        self,
        layer_dist_st: List[torch.Tensor], # shape: [batch_size_source X batch_size_target]
        layer_mean_st: List[torch.Tensor], # shape: [batch_size_source X batch_size_target]
        class_index: Optional[int]=None
    ):
        cum_kernel_dist = None
        for i, order, multiplier in zip(
            range(self.num_layers),
            self.kernel_orders,
            self.kernel_multipliers
        ):
            distance = layer_dist_st[i] if class_index is None else layer_dist_st[i][class_index]
            mean = layer_mean_st[i] if class_index is None else layer_mean_st[i][class_index]
            
            kernel_dist = self.compute_kernel_dist(distance, mean, order, multiplier)
            if cum_kernel_dist is None:
                cum_kernel_dist = kernel_dist
            else:
                cum_kernel_dist += kernel_dist
                
        return cum_kernel_dist
      
    def compute_kernel_dist(
        self,
        distance: torch.Tensor, # shape: [batch_size_source X batch_size_target]
        mean: torch.Tensor, # shape: [batch_size_source X batch_size_target]
        kernel_order: int,
        kernel_multiplier: int
    ):
        base_mean = mean / (kernel_multiplier ** (kernel_order // 2))
        mean_list = [base_mean * (kernel_multiplier**i) for i in range(kernel_order)]
        mean_tensor = torch.stack(mean_list, dim=0).to(self.device)

        eps = 1e-5
        mean_mask = (mean_tensor < eps).type(torch.FloatTensor)
        mean_tensor = (1.0 - mean_mask) * mean_tensor + mean_mask * eps 
        mean_tensor = mean_tensor.detach()

        for i in range(len(mean_tensor.size()) - len(distance.size())):
            distance = distance.unsqueeze(0)

        distance = distance / mean_tensor
        upper_mask = (distance > 1e5).type(torch.FloatTensor).detach()
        lower_mask = (distance < 1e-5).type(torch.FloatTensor).detach()
        normal_mask = 1.0 - upper_mask - lower_mask
        distance = normal_mask * distance + upper_mask * 1e5 + lower_mask * 1e-5
        kernel_val = torch.sum(torch.exp(-1.0 * distance), dim=0)
        return kernel_val
    
    def classes_mean_kernel(
        self,
        samples_for_class_source: List[int], # shape: [num_classes] 
        samples_for_class_target: List[int], # shape: [num_classes] 
        cumulate_kernel: torch.Tensor # shape: [batch_size_source X batch_size_target]
    ) -> torch.Tensor: # shape: [num_classes X num_classes]
        mean = torch.zeros(
            (len(samples_for_class_source), len(samples_for_class_source))
        ).to(self.device)
        x_start = 0
        x_end = 0
        for source_class_index in range(len(samples_for_class_source)):
            x_start = x_end
            x_end = x_start + samples_for_class_source[source_class_index]
            
            y_start = 0
            y_end = 0
            for target_class_index in range(len(samples_for_class_target)):
                y_start = y_end
                y_end = y_start + samples_for_class_target[target_class_index]
                
                class_mean = torch.mean(
                    cumulate_kernel[
                        x_start:x_start + samples_for_class_source[source_class_index],
                        y_start:y_start + samples_for_class_target[target_class_index]
                    ]
                )
                mean[source_class_index, target_class_index] = class_mean
        return mean

## Models

All models are composed by three main elements:

- backbone
- feature processor
- classifier

The backbone is a pretrained network from `torchvision` trained with a smaller learning rate than the one used for other two components.
The feature extractor is used for process the output of the backbone, we decided to separate these two steps into two classes in order to have more control on both of them, this allow to make tests with many combinations.
The last element is the classifier, it is a simple linear layer that takes the output of backbone (feature processor if it is used) and return an array of prediction with the same length of the number of classes.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision import models

from typing import Optional

In [ ]:
class Backbone(nn.Module):
    def __init__(
        self,
        backbone_type: str,
    ) -> None:
        super(Backbone, self).__init__()
        if backbone_type == "resnet34":
            self.backbone = models.resnet34(pretrained=True)
            self.output_num = self.backbone.fc.in_features
            self.backbone.fc = nn.Sequential()
        else:
            raise ValueError("select one valid backbone_type")
            
        
    def get_output_num(self):
        return self.output_num
        
    def forward(self, x):
        return self.backbone(x)
    
class FeatureProcessor(nn.Module):
    def __init__(self, input_size: int, output_size: int) -> None:
        super(FeatureProcessor, self).__init__()
        self.processing_layer = nn.Sequential(
            nn.Linear(input_size, output_size),
            nn.ReLU()
        )
        self.output_num = output_size
        
    def get_output_num(self):
        return self.output_num
        
    def forward(self, x):
        return self.processing_layer(x)
    
class Classifier(nn.Module):
    def __init__(
        self,
        backbone_type: str,
        n_classes: int,
        n_feature_extracted: Optional[int] = None
    ) -> None:
        super(Classifier, self).__init__()
        self.backbone = Backbone(backbone_type)
        if n_feature_extracted:
            self.processing_features = True
            self.processing_layer = FeatureProcessor(self.backbone.get_output_num(), n_feature_extracted)
        else:
            self.processing_features = False
        self.classification_layer = nn.Linear(
            self.processing_layer.get_output_num() if self.processing_features else self.backbone.get_output_num(),
            n_classes
        )
        self.output_num = n_classes
        
    def get_output_num(self):
        self.output_num
        
    def get_features(self, x):
        features = self.backbone(x)
        if self.processing_features:
            features = self.processing_layer(features)
        return features
        
    def forward(self, x):
        features = self.get_features(x)
        classes = self.classification_layer(features)
        
        return features, classes

## Dataset

In [ ]:
from torchvision.datasets import ImageFolder
from torchvision import transforms as T

from torch.utils.data import DataLoader, random_split

In [ ]:
source_dataset = ImageFolder(
    DATASET_PATH / "product_images",
    transform=T.Compose([
        T.Resize((256, 256)),
        T.CenterCrop((224, 224)),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
)
target_dataset = ImageFolder(
    DATASET_PATH / "real_life",
    transform=T.Compose([
        T.Resize((256, 256)),
        T.CenterCrop((224, 224)),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
)

validation_ratio = .2
test_ratio = .2

validation_dataset, test_dataset, train_dataset = random_split(
    source_dataset,
    [
        int(len(source_dataset)*validation_ratio),
        int(len(source_dataset)*test_ratio),
        len(source_dataset) - (
            int(len(source_dataset)*validation_ratio)
            +int(len(source_dataset)*test_ratio)
        )
    ]
)

target_test_ratio = .4
target_test_dataset, target_train_dataset = random_split(
    target_dataset,
    [
        int(len(target_dataset)*target_test_ratio),
        len(target_dataset) - int(len(target_dataset)*target_test_ratio)
    ]
)

In [ ]:
len(train_dataset), len(validation_dataset), len(test_dataset), len(target_train_dataset), len(target_test_dataset)

## Train

In [ ]:
from torch.optim import lr_scheduler, SGD, Adam

from tqdm import tqdm
#import torchinfo

In [ ]:
# misc
set_random_seed(33)
device = get_device()
num_threads = 4

# train
num_epochs = 30
batch_size = 32
lr = 0.001
weight_decay = 0.000001
momentum = 0.9
scheduler_factor = 0.25
scheduler_patience = 20

# domain adaptation loss
use_coral = True
coral_weight = 20

In [ ]:
def get_data_loader(dataset, batch_size, num_threads):
    return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_threads
    )

def get_optimizer(model, lr, wd, momentum):
    higher_lr_weights = []
    lower_lr_weights = []

    for name, param in model.named_parameters():
        if name.startswith('backbone'):
            lower_lr_weights.append(param)
        elif name.startswith('processing') or name.startswith('classification'):
            higher_lr_weights.append(param)
        else:
            raise ValueError(f"Unexpected type of weights {name}")

    optimizer = torch.optim.SGD(
        [
            {'params': lower_lr_weights},
            {'params': higher_lr_weights, 'lr': lr}
        ],
        lr=lr / 10,
        weight_decay=wd,
        momentum=momentum
    )

    return optimizer

In [ ]:
        model = Classifier(
    backbone_type="resnet34", 
    n_classes=len(dataset.classes),
    # n_feature_extracted=2048
)
model.to(device)
optimizer = get_optimizer(model, lr, weight_decay, momentum)
criterion = nn.CrossEntropyLoss()
scheduler = lr_scheduler.ReduceLROnPlateau(
    optimizer,
    'min',
    factor=scheduler_factor,
    patience=scheduler_patience,
    verbose=True
) 
dataloaders = {
    'train': {
        'source': get_data_loader(train_dataset, batch_size, num_threads),
        'target': get_data_loader(target_train_dataset, batch_size, num_threads),
    },
    'validation': {
        'source': get_data_loader(validation_dataset, batch_size, num_threads),
    }
    #'test': get_data_loader(test_dataset, batch_size, num_threads)
}
#summary(model, input_size=(batch_size, *dataset[0][0].size()))

In [ ]:
best_model = model
best_loss = np.Inf

phases_loss = {k: [] for k in dataloaders.keys()}
phases_acc = {k: [] for k in dataloaders.keys()}
for epoch in range(num_epochs):
    for phase, phase_dataloaders in dataloaders.items():
        source_dataloader = phase_dataloaders['source']
        target_dataloader = phase_dataloaders.get('target', None)
        if phase == "train":
            model.train()
        else:
            model.eval()

        epoch_loss = 0.0
        epoch_acc = 0.0
        for index, (x, labels) in enumerate(source_dataloader):
            x = x.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()

            with torch.autocast(device_type=device.type):
                with torch.set_grad_enabled(phase == "train"):
                    features, predictions = model(x)
                    x = x.detach().cpu()
                    
                    loss = criterion(predictions, labels)
                    predictions = predictions.detach().cpu()
                    labels = labels.detach().cpu()
                    if use_coral and not target_dataloader is None:
                        source_feauteres = features.cpu()
                        target_x, _ = next(target_dataloader._get_iterator())
                        target_x = target_x.to(device)
                        target_fuatures, _ = model(target_x)
                        target_x = target_x.detach().cpu()
                        target_fuatures = target_fuatures.cpu()
                        loss = loss + coral_weight * coral(source_feauteres, target_fuatures)
                    epoch_loss += loss.item()
                    epoch_acc += (torch.argmax(predictions, 1) == labels).sum().item() / labels.size()[0]

                    if phase == "train":
                        loss.backward()
                        optimizer.step()

        epoch_loss /= len(source_dataloader)
        epoch_acc /= len(source_dataloader)
        phases_loss[phase].append(epoch_loss)
        phases_acc[phase].append(epoch_acc)

        if phase == "validation" and abs(epoch_loss) <= abs(best_loss):
            best_model = model
            best_loss = epoch_loss
            scheduler.step(epoch_loss)
            
    print_epoch_chart(
        phases_loss['train'],
        phases_loss['validation'],
        phases_acc['train'],
        phases_acc['validation'],
        epoch,
        num_epochs
    )

### Test model

In [ ]:
predictions = []
reals = []
for x, y in test_dataset:
    x_gpu = x.unsqueeze(0).to(device)
    _, predicted = best_model(x_gpu)
    predictions.append(np.argmax(predicted.detach().cpu().numpy()))
    reals.append(y)
    del x_gpu
predictions = np.array(predictions)
reals = np.array(reals)

In [ ]:
def plot_predictions(index, idx_to_class, dataset, reals, predictions):
    x, _ = dataset[index]
    plt.imshow(x[1])
    print(idx_to_class[reals[index]], idx_to_class[predictions[index]])

In [ ]:
plot_predictions(350, idx_to_class, test_dataset, reals, predictions) 

In [ ]:
(reals == predictions).sum() / reals.shape[0]

## Domain adaptation

### Plain

In [ ]:
predictions = []
reals = []
for x, y in target_dataset:
    x_gpu = x.unsqueeze(0).to(device)
    _, predicted = best_model(x_gpu)
    predictions.append(np.argmax(predicted.detach().cpu().numpy()))
    reals.append(y)
    del x_gpu
predictions = np.array(predictions)
reals = np.array(reals)

In [ ]:
def plot_predictions(index, idx_to_class, dataset, reals, predictions):
    x, _ = dataset[index]
    plt.imshow(x[1])
    print(idx_to_class[reals[index]], idx_to_class[predictions[index]])

In [ ]:
plot_predictions(350, idx_to_class, target_dataset, reals, predictions) 

In [ ]:
(reals == predictions).sum() / reals.shape[0]